**Building Application with OSS Models**
**SQLCoder-7b-2**

In [1]:
!pip install torch transformers bitsandbytes accelerate sqlparse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.1 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
torch.cuda.is_available()

True

In [5]:
available_memory = torch.cuda.get_device_properties(0).total_memory
print(available_memory)

15835660288


Download the Model

In [9]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 16e9:
  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      trust_remote_code=True,
      device_map="auto",
      torch_dtype=torch.float16,
      use_cache=True
      )
else:
  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      trust_remote_code=True,
      load_in_8bit=True,
      device_map="auto",
      torch_dtype=torch.float16,
      use_cache=True
      )

tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Set the Question & Prompt and Tokenize

In [12]:
prompt = """### Task
Generate a SQL query to answer [Question]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I don't know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
CREATE TABLE products (
  product_id INTeger PRIMARY KEY, -- Unique identifier for each product
  product_name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INT -- Quantity in stock
);

CREATE TABLE customers (
  customer_id INTEGER PRIMARY KEY, -- Unique identifier for each customer
  name VARCHAR(50), -- Name of the customer
  address VARCHAR(100), -- Address of the customer
);

CREATE TABLE salepeople (
  salepeople_id INTEGER PRIMARY KEY, -- Unique identifier for each salepeople
  name VARCHAR(50), -- Name of the salepeople
  region VARCHAR(50) -- Region of the salepeople
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique identifier for each sale
  product_id INTEGER, -- Foreign key referencing the products table
  customer_id INTEGER, -- Foreign key referencing the customers table
  salepeople_id INTEGER, -- Foreign key referencing the salepeople
  sale_date DATE, -- Date of the sale
  quantity INTEGER, -- Quantity of the product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique identifier for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price of the product
):

-- sales.product_id can be used to join with products.product_id
-- sales.customer_id can be used to join with customers.customer_id
-- sales.salepeople_id can be used to join with salepeople.salepeople_id
-- product_suppliers.product_id can be used to join with products.product_id

### Answer
Given the database schema, here is the SQL query to answer [Question]{question}[/QUESTION]
[SQL]
"""

Generate the SQL

In [15]:
import sqlparse

def generate_query(question):
  updated_prompt = prompt.format(question=question)
  inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
  generated_ids = model.generate(
      **inputs,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      pad_token_id=tokenizer.eos_token_id,
      max_new_tokens=200,
      do_sample=True,
      num_beams = 1,
  )
  outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
  torch.cuda.empty_cache()
  torch.cuda.synchronize()
  return sqlparse.format(outputs[0].split("[SQL]")[-1],reindent=True)


In [16]:
question = "What was the highest quantity sold last month?"
generate_sql = generate_query(question)
print(generate_sql)


SELECT MAX(s.quantity) AS max_quantity
FROM sales s
WHERE s.sale_date >= (CURRENT_DATE - interval '1 month');


In [17]:
question = "Which salesperson sold large amount of products last month?"
generate_sql = generate_query(question)
print(generate_sql)


SELECT s.salepeople_id,
       SUM(s.quantity) AS total_quantity_sold
FROM sales s
WHERE s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY s.salepeople_id
ORDER BY total_quantity_sold DESC
LIMIT 1;


In [18]:
question = "What was our revenue by product in the new york last month?"
generate_sql = generate_query(question)
print(generate_sql)


SELECT p.product_name,
       SUM(s.quantity * p.price) AS revenue
FROM sales s
JOIN product_suppliers ps ON s.product_id = ps.product_id
JOIN products p ON ps.product_id = p.product_id
JOIN salepeople sp ON s.salepeople_id = sp.salepeople_id
WHERE sp.region = 'new york'
  AND s.sale_date >= (CURRENT_DATE - interval '1 month')
GROUP BY p.product_name;
